# Demonstrate ``nscan``

In this example, we demonstrate the `apstools.plans.nscan()` plan.  The `nscan()` plan scans over ``n`` variables moved together, each in equally spaced steps.

Here, we scan two motors together, each in equally spaced steps.  We use an [`swait`](https://htmlpreview.github.io/?https://raw.githubusercontent.com/epics-modules/calc/R3-6-1/documentation/swaitRecord.html) record (part of the *userCalc* support from synApps) as a detector.  We configure the `swait` record with a calculation (from `apstools.plans.swait_setup_random_number()`) that computes a noisy (random number) signal.  

For this demo, we do not need the databroker since we do not plan to review any of this data after collection.  We'll display the data during the scan using the *LiveTable()* code.

In [1]:
from ophyd import EpicsMotor
from bluesky.callbacks import LiveTable
from apstools.synApps_ophyd import swaitRecord, swait_setup_random_number
from apstools.plans import nscan

from bluesky import RunEngine
RE = RunEngine({})

Figure out which workstation we are running.  The *mint-vm* host has a different IOC prefix.

In [2]:
import socket
if socket.gethostname().find("mint-vm") >= 0:
    prefix = "vm7:"
else:
    prefix = "xxx:"

Connect to our motors and create the *noisy* detector.

In [3]:
m1 = EpicsMotor(prefix+"m1", name="m1")
m2 = EpicsMotor(prefix+"m2", name="m2")
noisy = swaitRecord(prefix+"userCalc1", name="noisy")
# configure the *detector* as a random number generator using a calculation.
swait_setup_random_number(noisy)

Measure the noisy detector while step scanning both the m1 & m2 motors together.  We'll move m2 twice as far as m1, like a $\theta$-$2\theta$ scan on a diffractometer.

In [4]:
RE(
    nscan([noisy, ], m1, 2, 0, m2, 4, 0, num=6),
    LiveTable(["m1", "m2", "noisy_val"])
    )

+-----------+------------+------------+------------+------------+
|   seq_num |       time |         m1 |         m2 |  noisy_val |
+-----------+------------+------------+------------+------------+
|         1 | 15:43:39.7 |    2.00000 |    4.00000 |    0.00000 |
|         2 | 15:43:40.8 |    1.60000 |    3.20000 |    0.00000 |
|         3 | 15:43:41.9 |    1.20000 |    2.40000 |    0.00000 |
|         4 | 15:43:43.0 |    0.80000 |    1.60000 |    0.00000 |
|         5 | 15:43:44.2 |    0.40000 |    0.80000 |    0.00000 |
|         6 | 15:43:45.2 |    0.00000 |    0.00000 |    0.00000 |
+-----------+------------+------------+------------+------------+
generator nscan ['09e401ce'] (scan num: 1)


('09e401ce-f4de-4230-af83-ed893a2c8184',)